In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


Build BioMistral Medical RAG Chatbot using BioMistral Open Source LLM

In [ ]:
!pip install langchain sentence-transformers chromadb llama-cpp-python langchain_community pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 MB 11.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94

In [ ]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain_community.llms import llamacpp
from langchain.chains import RetrievalQA, LLMChain
import os

In [ ]:
from llama_cpp import Llama

llm = Llama.from_pretrained(
	repo_id="LoneStriker/BioMistral-7B-DARE-GGUF",
	filename="BioMistral-7B-DARE-Q3_K_L.gguf",
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


BioMistral-7B-DARE-Q3_K_L.gguf:   0%|          | 0.00/3.82G [00:00<?, ?B/s]

llama_model_loader: loaded meta data with 23 key-value pairs and 291 tensors from /root/.cache/huggingface/hub/models--LoneStriker--BioMistral-7B-DARE-GGUF/snapshots/f6ada0eb94033e31f0574a5a06ac96ba9536cf9d/./BioMistral-7B-DARE-Q3_K_L.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = models
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llam

In [ ]:
loader=PyPDFDirectoryLoader("/content/Project")
docs=loader.load()

In [ ]:
 len(docs)

95

**Chunking**

In [ ]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks=text_splitter.split_documents(docs)

In [ ]:
len(chunks)

585

In [ ]:
chunks[500]

Document(metadata={'source': '/content/Project/healthyheart.pdf', 'page': 79}, page_content='Stay the course. Unless you have to stop your activity for a health\nreason, stick with it. If you feel like giving up because you think\nyou’re not going as fast or as far as you should, set smaller, short-\nterm goals for yourself. If you find yourself becoming bored, try')

##Embeddings Creations

In [ ]:
os.environ["HUGGINGFACEHUB_API_TOKEN"]="Your API KEY"

In [ ]:
embeddings=SentenceTransformerEmbeddings(model_name="NeuML/pubmedbert-base-embeddings")

<ipython-input-20-059dc5d344e3>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=SentenceTransformerEmbeddings(model_name="NeuML/pubmedbert-base-embeddings")
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.12k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/706k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

##Vector Store Creation

In [ ]:
vectorstore=Chroma.from_documents(chunks,embeddings)

INFO:backoff:Backing off send_request(...) for 0.8s (posthog.request.APIError: [PostHog] upstream connect error or disconnect/reset before headers. reset reason: overflow (503))
INFO:backoff:Backing off send_request(...) for 0.6s (posthog.request.APIError: [PostHog] upstream connect error or disconnect/reset before headers. reset reason: overflow (503))


In [ ]:
query="what is the age of heart disease?"

search_results=vectorstore.similarity_search(query)

In [ ]:
search_results

[Document(metadata={'page': 5, 'source': '/content/Project/healthyheart.pdf'}, page_content='Consider these facts: \n■ Each year, 500,000 Americans die of heart disease, and approx-\nimately half of them are women.\n■ As early as age 45, a man’s risk of heart disease begins to rise \nsignificantly. For a woman, risk starts to increase at age 55.'),
 Document(metadata={'page': 5, 'source': '/content/Project/healthyheart.pdf'}, page_content='If you’re a man, you may think you’re not old enough to have a\nserious heart condition.\nWrong on all counts. In the United States, heart disease is the #1\nkiller of both women and men. It affects many people at midlife, \nas well as in old age. It also can happen to those who “feel fine.”'),
 Document(metadata={'page': 8, 'source': '/content/Project/healthyheart.pdf'}, page_content='menopause, women are more likely to develop heart disease. For\nboth women and men, middle age is a time of increasing risk\nbecause people are more likely to develop 

In [ ]:
retriever=vectorstore.as_retriever(search_kwargs={"k":5})

In [ ]:
retriever.get_relevant_documents(query)

<ipython-input-25-162c3489abc9>:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents(query)


[Document(metadata={'page': 5, 'source': '/content/Project/healthyheart.pdf'}, page_content='Consider these facts: \n■ Each year, 500,000 Americans die of heart disease, and approx-\nimately half of them are women.\n■ As early as age 45, a man’s risk of heart disease begins to rise \nsignificantly. For a woman, risk starts to increase at age 55.'),
 Document(metadata={'page': 5, 'source': '/content/Project/healthyheart.pdf'}, page_content='If you’re a man, you may think you’re not old enough to have a\nserious heart condition.\nWrong on all counts. In the United States, heart disease is the #1\nkiller of both women and men. It affects many people at midlife, \nas well as in old age. It also can happen to those who “feel fine.”')]

##LLM Model loading


In [ ]:
from llama_cpp import Llama

llm = Llama.from_pretrained(
	repo_id="LoneStriker/BioMistral-7B-DARE-GGUF",
	filename="BioMistral-7B-DARE-Q3_K_L.gguf",
  temperature=0.2,
  max_tokens=2048,
  top_p=3
)

llama_model_loader: loaded meta data with 23 key-value pairs and 291 tensors from /root/.cache/huggingface/hub/models--LoneStriker--BioMistral-7B-DARE-GGUF/snapshots/f6ada0eb94033e31f0574a5a06ac96ba9536cf9d/./BioMistral-7B-DARE-Q3_K_L.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = models
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llam

Use LLM and retriver and query, to generate final response

In [ ]:
template="""
<|context|>
You are an Medical Assistant that follows the instructions and generate the accurate response based on the query and the context provided.
Please be truthful and give direct answers.
</s>
<|user|>
{query}
</s>
<|assistant|>
"""


In [ ]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import ChatPromptTemplate

In [ ]:
prompt=ChatPromptTemplate.from_template(template)

In [ ]:
rag_chain=(
    {"context":retriever, "query":RunnablePassthrough()}
    |prompt
    |llm
    |StrOutputParser()
)

In [ ]:
# RAG Chain (combining retriever, prompt, and model)
def rag_chain(query, retriever):
    # Retrieve the context
    context = retriever  # Replace with your actual retriever logic

    # Format the prompt with the retrieved context and query
    formatted_prompt = prompt.format(context=context, query=query)

    # Use the Llama model to generate a response
    response = llm(formatted_prompt)

    # Parse the output (if needed, based on the model's response structure)
    parsed_response = response["choices"][0]["text"]  # Adjust this based on your Llama's response format

    return parsed_response


In [ ]:
# Example query
query = "who is the risk of diabetes?"
retriever = "Diabetes symptoms include increased thirst, frequent urination, and fatigue."  # Replace with actual retrieval
response = rag_chain(query, retriever)
print(response)

Llama.generate: 71 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =   34939.02 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    16 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   11278.33 ms /    17 tokens


The risk of developing type 2 diabetes increases as you age, especially after the


In [ ]:
response

'Risk factors for type 2 diabetes include being overweight or obese,'

In [ ]:
import sys

while True:
    user_input = input("Input query: ")
    if user_input.lower() == "exit":
        print("Exiting...")
        sys.exit()
    if user_input == "":
        continue

    # Call the rag_chain function directly
    result = rag_chain(user_input, retriever="The heart is an organ that pumps blood throughout the body.")  # Example retriever
    print("Answer:", result)

Llama.generate: 54 prefix-match hit, remaining 19 prompt tokens to eval
llama_perf_context_print:        load time =   34939.02 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    19 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    13 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   20966.93 ms /    32 tokens


Answer: The average age of heart attack is 65 years old.
